In [ ]:
from google.colab import files
uploaded = files.upload()

Saving archive (4).zip to archive (4).zip


In [ ]:
import zipfile
import os

zpath = "/content/archive (4).zip"
epath = "/content/HandGesture"

with zipfile.ZipFile(zpath, 'r') as zip_ref:
  zip_ref.extractall(epath)

print("Extracted files:",os.listdir(epath))

Extracted files: ['HandGesture']


In [ ]:
dpath = "/content/HandGesture/HandGesture/images"
print("Subdirectories:", os.listdir(dpath))


Subdirectories: ['fingers_crossed', 'up', 'call_me', 'paper', 'scissor', 'peace', 'thumbs', 'okay', 'rock_on', 'rock']


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (128,128)
batch_size = 32
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)

validatagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

train_data = train_datagen.flow_from_directory(
    dpath,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training'
)

validata = validatagen.flow_from_directory(
    dpath,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'validation'
)

Found 4199 images belonging to 10 classes.
Found 1044 images belonging to 10 classes.


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3),activation='relu',input_shape=(128,128,3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dropout(0,5),
    layers.Dense(10,activation='softmax')
])

model.compile(optimizer='adam',loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,930 (12.61 MB)

 Trainable params: 3,305,930 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

earlystoppings = EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)
history = model.fit(
    train_data,
    epochs=10,
    validation_data=validata
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.2407 - loss: 2.0506 - val_accuracy: 0.6341 - val_loss: 0.9830
Epoch 2/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.6183 - loss: 1.0921 - val_accuracy: 0.8429 - val_loss: 0.4617
Epoch 3/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.7351 - loss: 0.7992 - val_accuracy: 0.8678 - val_loss: 0.3904
Epoch 4/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.7830 - loss: 0.6325 - val_accuracy: 0.9157 - val_loss: 0.2936
Epoch 5/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.8298 - loss: 0.5298 - val_accuracy: 0.9071 - val_loss: 0.2937
Epoch 6/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.8436 - loss: 0.4904 - val_accuracy: 0.9301 - val_loss: 0.2189
Epoch 7/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 155s 1s/step - accuracy: 0.8603 - loss: 0.4179 - val_accuracy: 0.9368 - val_loss: 0.1899
Epoch 8/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 208s 1s/step - accuracy: 0.8841 - loss: 0.3642 - val_accu